In [1]:
# imports
import requests
from googleapiclient.discovery import build
import google.datalab.storage as storage
from PIL import Image, ImageDraw
from StringIO import StringIO
from io import BytesIO
import base64

In [171]:
#imports after the fact

In [2]:
# API key
APIKEY="IzaSyDvN7_zfln0Mrxx0MWWoJGeGbWqasQiinc"

In [ ]:
############################################################################################### 
#
#  NOTE - the only thing to customize is the names below; after the files are written
#   back to a cloud storage bucket; they will need to be decoded to be viewed
#
###############################################################################################

In [81]:
# manipulate file names to create in and out file
def return_file_names(image):
    in_file = "gs://machine-learning-backend-storagebucket/" + image
    img_out = image.rsplit('/',1)[0] + "/output/" + image.rsplit('/',1)[1] + "_bound_box.png"
    text_out = image.rsplit('/',1)[0] + "/output/" + image.rsplit('/',1)[1] + "_text.txt"
    img_in = image
    return in_file, img_out, img_in, text_out

In [82]:
def make_api_call(image):
    vservice = build('vision', 'v1', developerKey=APIKEY)
    request = vservice.images().annotate(body={
            'requests': [{
                    'image': {
                        'source': {
                            'imageUri': image
                        }
                    },
                    'features': [{
                        'type': 'DOCUMENT_TEXT_DETECTION',
                        'maxResults': 1,
                    }]
                }],
            })
    responses = request.execute(num_retries=3)
    pages = responses['responses'][0]['fullTextAnnotation']['pages']
    text = responses['responses'][0]['fullTextAnnotation']['text']
    return pages, text

In [62]:
def draw_boxes(image, blocks, color):
    """Draw a border around the image using the hints in the vector list."""
    draw = ImageDraw.Draw(image)

    for block in blocks:
        # upper left
        try:
            upper_left_x = block['vertices'][0]['x']
        except KeyError:
            upper_left_x = 0
        try:
            upper_left_y = block['vertices'][0]['y']
        except KeyError:
            upper_left_y = 0
        # upper right
        try:
            upper_right_x = block['vertices'][1]['x']
        except KeyError:
            upper_right_x = 0
        try:
            upper_right_y = block['vertices'][1]['y']
        except KeyError:
            upper_right_y = 0

        # lower right
        try:
            lower_right_x = block['vertices'][2]['x']
        except KeyError:
            lower_right_x = 0
        try:
            lower_right_y = block['vertices'][2]['y']
        except KeyError:
            lower_right_y = 0
        # lower left
        try:
            lower_left_x = block['vertices'][3]['x']
        except KeyError:
            lower_left_x = 0
        try:
            lower_left_y = block['vertices'][3]['y']
        except KeyError:
            lower_left_y = 0

        upper_left = (upper_left_x, upper_left_y)
        upper_right = (upper_right_x, upper_right_y)
        lower_right = (lower_right_x, lower_right_y)
        lower_left = (lower_left_x, lower_left_y)
        draw.polygon([upper_left, upper_right, lower_right, lower_left], None, color)
    return image

In [63]:
def get_block_bounds(pages):    
    bounds = []
    for page in pages:
        for block in page['blocks']:
            bounds.append(block['boundingBox'])
    return bounds

def get_para_bounds(pages):    
    bounds = []
    for page in pages:
        for block in page['blocks']:
            for paragraph in block['paragraphs']:
                bounds.append(paragraph['boundingBox'])
    return bounds

def get_word_bounds(pages):    
    bounds = []
    for page in pages:
        for block in page['blocks']:
            for paragraph in block['paragraphs']:
                for word in paragraph['words']:
                    bounds.append(word['boundingBox'])
    return bounds

In [77]:
def render_doc_markup(pages, img_data_in, img_data_out):    
    img = Image.open( StringIO(img_data_in.read_stream()) )

    bounds = get_block_bounds(pages)
    draw_boxes(img, bounds, 'blue')
    
    bounds = get_para_bounds(pages)
    draw_boxes(img, bounds, 'green')

    bounds = get_word_bounds(pages)
    draw_boxes(img, bounds, 'orange')

    in_mem_file = BytesIO()
    img.save(in_mem_file, format = "PNG")
    # reset file pointer to start
    in_mem_file.seek(0)
    img_bytes = in_mem_file.read()

    base64_encoded_result_bytes = base64.b64encode(img_bytes)
    img_data_out.write_stream(base64_encoded_result_bytes, 'image/png')

In [75]:
def render_doc_text(text, text_data_out):
    text_data_out.write_stream(text.encode('ascii','ignore'), 'text/plain')

In [67]:
############################################################################################### 
#
#  PROGRAM - Start document markup
#
############################################################################################### 
# select the storage bucket object you want
images = list()
my_bucket = storage.Bucket('machine-learning-backend-storagebucket')
for obj in my_bucket.objects():
    if obj.key.find("Oxygen") > -1 and obj.key.find(".txt") == -1:
        images.append(obj.key)

In [83]:
for image in images:
    # create in and out names
    filein, img_out, img_in, txt_out = return_file_names(image)

    # open the input image file
    img_data_in  = my_bucket.object(img_in)
    img_data_out = my_bucket.object(img_out)
    text_data_out = my_bucket.object(txt_out)

    # call the API
    pages, text = make_api_call(filein)

    # render the document - comment out either of the following if desired
    render_doc_text(text, text_data_out)
    render_doc_markup(pages, img_data_in, img_data_out)

In [ ]:
############################################################################################### 
#
#  NOTE - run this locally to view the images
#
############################################################################################### 
import base64

with open("C:\\Users\\adpilz\\Pictures\\picturetester50.png", "rb") as image_file:
    with open("C:\\Users\\adpilz\\Pictures\\picturetester51.png", 'wb') as f:
        f.write(base64.b64decode(image_file.read()))